In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib
import re
import nltk
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 
from SentimentAnalyser import SentimentAnalyser
from nltk.tokenize.treebank import TreebankWordDetokenizer

## Read in data 

### Market data

In [2]:
btc_market = pd.read_csv("../data/Market/BTC-USD-80.csv")
btc_market.head()

,time,close,high,low,open,volumefrom,volumeto,mid
0,1589572800,9247.26,9398.00,9149.13,9341.23,7890.76,73070842.81,9273.565
1,1589576400,9416.95,9442.75,9209.07,9247.26,3556.95,33204441.74,9325.910
2,1589580000,9360.45,9481.70,9349.47,9416.95,3168.34,29846247.61,9415.585
3,1589583600,9311.69,9387.99,9260.10,9360.45,2593.46,24173364.88,9324.045
4,1589587200,9389.18,9392.76,9224.32,9311.69,3187.38,29687521.08,9308.540


#### Convert unix code time to more readable datetime format

In [3]:
btc_market['time_formatted'] = pd.to_datetime(btc_market['time'], unit='s')

In [4]:
btc_market[['time', 'time_formatted']].head()

,time,time_formatted
0,1589572800,2020-05-15 20:00:00
1,1589576400,2020-05-15 21:00:00
2,1589580000,2020-05-15 22:00:00
3,1589583600,2020-05-15 23:00:00
4,1589587200,2020-05-16 00:00:00


### Volatility data

In [5]:
btc_volatility = pd.read_csv("../data/Market/vcrix.csv")
btc_volatility.tail()

,date,vcrix
147,2020-05-27,873.952874
148,2020-05-28,871.938590
149,2020-05-29,900.197688
150,2020-05-30,898.384012
151,2020-05-31,819.182235


In [6]:
btc_volatility['date'] = pd.to_datetime(btc_volatility['date'])

In [7]:
# btc_volatility.drop(columns=['Unnamed: 0', 'n'], inplace=True)

# btc_volatility = btc_volatility[(btc_volatility['date'] >= '2020-01-01') & (btc_volatility['date'] < '2020-0-01')]
# btc_volatility

In [8]:
btc_volatility['vcrix_log_diff'] = np.log(btc_volatility['vcrix']) - np.log(btc_volatility['vcrix'].shift(1))

### Trend

In [9]:
btc_trend = pd.read_csv('../data/Trends/BTC_trend_complete.csv')
btc_trend

,date,Bitcoin,BTC,isPartial
0,2020-01-01,49,13,False


In [10]:
btc_trend['date'] = pd.to_datetime(btc_trend['date'])
btc_trend['date'].head()

0   2020-01-01
Name: date, dtype: datetime64[ns]

### Tweets

In [11]:
# sentiment analyser class
sent_analyser = SentimentAnalyser()

In [12]:
# stopwords for cleaning
stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])

In [13]:
# cleaning method for tweets
def processTweet(tweet):
        tweet = tweet.lower() # convert text to lower-case
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
        tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
        tweet = word_tokenize(tweet) # remove repeated characters (helloooooooo into hello)
        tweet = [word for word in tweet if word not in stopwords]
        tweet = TreebankWordDetokenizer().detokenize(tweet)
        return tweet

In [14]:
def processTweets(chunk):
    processed_tweets = []
    tweet_sentiments = []
    for tweet in chunk['text']:
        if isinstance(tweet, str):
            # clean tweet
            processed_tweet = processTweet(tweet)
            
            # perform sentiment analysis on cleaned tweet using Sentiment Analyser
            tweet_sentiment = sent_analyser.sentiment_analyzer_scores(processed_tweet) # returns dictionary 

            # add to respective lists
            processed_tweets.append(processed_tweet)
            tweet_sentiments.append(tweet_sentiment)

    return (processed_tweets, pd.DataFrame(tweet_sentiments))

In [15]:
# read the large csv file with specified chunksize 
df_chunk = pd.read_csv('../data/Tweets/combined_tweets.csv', chunksize=50000) # add nrows=100 for testing

#df_chunk = pd.read_csv('data/by_day/tweets_btc_2020-05-12.csv', chunksize=500)

In [16]:
chunk_list = []  # append each chunk df here 

# Each chunk is in df format
for chunk in df_chunk:  
    # drop columns where text contains nan
    chunk.dropna(subset=['text'], inplace=True)

    # init empty sentiment columns 
    chunk[['neg', 'neu', 'pos', 'compound']] = pd.DataFrame([[np.nan, np.nan, np.nan, np.nan]], index=chunk.index)

    # perform data filtering 
    chunk_text_filtered = processTweets(chunk)

    # set filtered text and sentiments
    chunk['text'], chunk[['neg', 'neu', 'pos', 'compound']] = chunk_text_filtered
    
    # Once the data filtering is done, append the chunk to list
    chunk_list.append(chunk)
    
# concat the list into dataframe 
btc_tweets = pd.concat(chunk_list.copy())

In [17]:
btc_tweets.head()

,screen_name,username,user_id,tweet_id,tweet_url,timestamp,timestamp_epochs,text,text_html,links,...,retweets,replies,is_replied,is_reply_to,parent_tweet_id,reply_to_users,neg,neu,pos,compound
0,bitcointe,Bitcointe 💎,1111726282008641542,1212523680120627200,/bitcointe/status/1212523680120627200,2020-01-01 23:59:07,1577923147,key gamers crypto blockchain share new 12 mont...,"<p class=""TweetTextSize js-tweet-text tweet-te...",['https://www.bitcointe.com/2020/01/01/key-gam...,...,0,0,False,False,NaN,[],0.000,0.774,0.226,0.3612
1,reqBitcoinJS,Bitcoin.js //Bitcoin price every half hour,1082323663238713345,1212523651477786624,/reqBitcoinJS/status/1212523651477786624,2020-01-01 23:59:00,1577923140,"jan 1 2020 23:58:00 utc 7,188 usd €6,409 eur £...","<p class=""TweetTextSize js-tweet-text tweet-te...",[],...,0,0,False,False,NaN,[],0.000,1.000,0.000,0.0000
2,Zectro1,Zectro,1035701087167102976,1212523515632586756,/Zectro1/status/1212523515632586756,2020-01-01 23:58:28,1577923108,'ve got divorce idea every critic bsv fan btcn...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],...,0,1,True,True,1.212506e+18,"[{'screen_name': '1071_twetch', 'user_id': '97...",0.293,0.481,0.225,-0.4534
3,domainrecap,Domain Recap,938629455446200325,1212523382463438849,/domainrecap/status/1212523382463438849,2020-01-01 23:57:56,1577923076,released tbr .ca results leftovers january 1 b...,"<p class=""TweetTextSize js-tweet-text tweet-te...",['https://www.domainrecap.com/2020/picks/to-be...,...,0,0,False,False,NaN,[],0.000,1.000,0.000,0.0000
4,BittrexPrices,BittrexPrices,1000111792444657665,1212522980854632450,/BittrexPrices/status/1212522980854632450,2020-01-01 23:56:20,1577922980,prices update usdt 1 hour btc 7198.97 +0.07 et...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],...,0,0,False,False,NaN,[],0.000,1.000,0.000,0.0000


In [23]:
btc_tweets['timestamp'] = pd.to_datetime(btc_tweets['timestamp'])

In [24]:
btc_tweets.sort_values(by="timestamp", inplace=True)

In [25]:
# check if duplicate tweets exist
len(btc_tweets['tweet_id']) != len(set(btc_tweets['tweet_id']))

True

In [26]:
# drop all rows with duplicate tweet id
btc_tweets.drop_duplicates(subset ="tweet_id", inplace = True)

# check again if duplicate tweets exist
len(btc_tweets['tweet_id']) != len(set(btc_tweets['tweet_id']))

False

In [27]:
btc_tweets.shape

(997, 25)

In [28]:
btc_tweets.head()

,screen_name,username,user_id,tweet_id,tweet_url,timestamp,timestamp_epochs,text,text_html,links,...,retweets,replies,is_replied,is_reply_to,parent_tweet_id,reply_to_users,neg,neu,pos,compound
1003,BinancePrices,BinancePrices,1000390115611566080,1212415970423259136,/BinancePrices/status/1212415970423259136,2020-01-01 16:51:07,1577897467,prices update usdt 1 hour btc 7227.23 +0.1 eth...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],...,0,0,False,False,NaN,[],NaN,NaN,NaN,NaN
1002,cryptosEUR,Cryptocurrencies / EUR,1025494047564226560,1212416074920157186,/cryptosEUR/status/1212416074920157186,2020-01-01 16:51:32,1577897492,prices update eur 1 hour btc 6436.2 € +0.04 et...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],...,0,0,False,False,NaN,[],0.0,1.000,0.000,0.0000
1001,cryptosUSD,Cryptocurrencies / USD,1024959511071797248,1212416083623325696,/cryptosUSD/status/1212416083623325696,2020-01-01 16:51:34,1577897494,prices update usd 1 hour btc 7209.1 +0.01 eth ...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],...,0,0,False,False,NaN,[],0.0,1.000,0.000,0.0000
1000,the_alphatrades,Alpha Trades,876494440369717248,1212416252523749376,/the_alphatrades/status/1212416252523749376,2020-01-01 16:52:14,1577897534,bitcoin bottomed 6400 opportunities local pric...,"<p class=""TweetTextSize js-tweet-text tweet-te...",['https://youtu.be/yXL1t3Qy07U'],...,0,0,False,False,NaN,[],0.0,1.000,0.000,0.0000
999,IAM__Network,IAM Platform,226310002,1212416348288159756,/IAM__Network/status/1212416348288159756,2020-01-01 16:52:37,1577897557,iam platform curated tweet istanbul blockchain...,"<p class=""TweetTextSize js-tweet-text tweet-te...",['https://blog.iammarketingmedia.com/istanbul-...,...,0,0,False,False,NaN,[],0.0,0.843,0.157,0.3818


### Merge two data frames

In [29]:
# first merge tweets with hourly trend data
btc_merged_hourly = pd.merge_asof(btc_tweets, btc_trend, left_on='timestamp', right_on='date', direction='backward')
btc_merged_hourly.head()

,screen_name,username,user_id,tweet_id,tweet_url,timestamp,timestamp_epochs,text,text_html,links,...,parent_tweet_id,reply_to_users,neg,neu,pos,compound,date,Bitcoin,BTC,isPartial
0,BinancePrices,BinancePrices,1000390115611566080,1212415970423259136,/BinancePrices/status/1212415970423259136,2020-01-01 16:51:07,1577897467,prices update usdt 1 hour btc 7227.23 +0.1 eth...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],...,NaN,[],NaN,NaN,NaN,NaN,2020-01-01,49,13,False
1,cryptosEUR,Cryptocurrencies / EUR,1025494047564226560,1212416074920157186,/cryptosEUR/status/1212416074920157186,2020-01-01 16:51:32,1577897492,prices update eur 1 hour btc 6436.2 € +0.04 et...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],...,NaN,[],0.0,1.000,0.000,0.0000,2020-01-01,49,13,False
2,cryptosUSD,Cryptocurrencies / USD,1024959511071797248,1212416083623325696,/cryptosUSD/status/1212416083623325696,2020-01-01 16:51:34,1577897494,prices update usd 1 hour btc 7209.1 +0.01 eth ...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],...,NaN,[],0.0,1.000,0.000,0.0000,2020-01-01,49,13,False
3,the_alphatrades,Alpha Trades,876494440369717248,1212416252523749376,/the_alphatrades/status/1212416252523749376,2020-01-01 16:52:14,1577897534,bitcoin bottomed 6400 opportunities local pric...,"<p class=""TweetTextSize js-tweet-text tweet-te...",['https://youtu.be/yXL1t3Qy07U'],...,NaN,[],0.0,1.000,0.000,0.0000,2020-01-01,49,13,False
4,IAM__Network,IAM Platform,226310002,1212416348288159756,/IAM__Network/status/1212416348288159756,2020-01-01 16:52:37,1577897557,iam platform curated tweet istanbul blockchain...,"<p class=""TweetTextSize js-tweet-text tweet-te...",['https://blog.iammarketingmedia.com/istanbul-...,...,NaN,[],0.0,0.843,0.157,0.3818,2020-01-01,49,13,False


In [30]:
btc_merged = pd.merge_asof(btc_merged_hourly, btc_volatility, left_on='timestamp', right_on='date', direction='backward', suffixes=("", "_vcrix"))

In [31]:
pd.set_option('display.max_colwidth', 50)

In [32]:
btc_merged.sample(n=12)

,screen_name,username,user_id,tweet_id,tweet_url,timestamp,timestamp_epochs,text,text_html,links,...,neu,pos,compound,date,Bitcoin,BTC,isPartial,date_vcrix,vcrix,vcrix_log_diff
395,CryptoRichYT,Crypto Rich,969952716930867202,1212456677187956736,/CryptoRichYT/status/1212456677187956736,2020-01-01 19:32:52,1577907172,banking unbanked interest free loans poor bloc...,"<p class=""TweetTextSize js-tweet-text tweet-te...",['https://bittubers.com/post/473158df-1184-4b3...,...,0.415,0.363,0.6249,2020-01-01,49,13,False,2020-01-01,564.354535,NaN
898,CoinMoonApp,CoinMoon,1176034667385872384,1212518608191901696,/CoinMoonApp/status/1212518608191901696,2020-01-01 23:38:58,1577921938,check top coins daily markets summary jan 2nd ...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],...,0.933,0.067,0.2023,2020-01-01,49,13,False,2020-01-01,564.354535,NaN
355,mr_kam_ogwu,ogwu osaemezu E,517479264,1212452530938093568,/mr_kam_ogwu/status/1212452530938093568,2020-01-01 19:16:24,1577906184,0.1 bitcoin btc're interested selling pls dm asap,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],...,0.600,0.400,0.4588,2020-01-01,49,13,False,2020-01-01,564.354535,NaN
182,CryptoGulp,Crypto Gulp,945716745192574977,1212433388268900353,/CryptoGulp/status/1212433388268900353,2020-01-01 18:00:20,1577901620,top 5 cryptocurrencies current prices bitcoin ...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],...,0.933,0.067,0.2023,2020-01-01,49,13,False,2020-01-01,564.354535,NaN
30,bitcoinagile,BitcoinAgile,2281314234,1212418404298215424,/bitcoinagile/status/1212418404298215424,2020-01-01 17:00:47,1577898047,alleged 8b btc tulip trust expires tales bonde...,"<p class=""TweetTextSize js-tweet-text tweet-te...",['https://news.bitcoin.com/alleged-8b-btc-tuli...,...,0.875,0.125,0.5106,2020-01-01,49,13,False,2020-01-01,564.354535,NaN
232,IAM__Network,IAM Platform,226310002,1212438938419449856,/IAM__Network/status/1212438938419449856,2020-01-01 18:22:23,1577902943,iam platform curated tweet china expands appli...,"<p class=""TweetTextSize js-tweet-text tweet-te...",['https://blog.iammarketingmedia.com/china-exp...,...,0.920,0.080,0.1027,2020-01-01,49,13,False,2020-01-01,564.354535,NaN
668,Bullrun_Gravano,𝐁𝐮𝐥𝐥𝐫𝐮𝐧 𝐆𝐫𝐚𝐯𝐚𝐧𝐨,999367950841806849,1212487893912039424,/Bullrun_Gravano/status/1212487893912039424,2020-01-01 21:36:55,1577914615,wow ...... seriously big news gcr...expecting ...,"<p class=""TweetTextSize js-tweet-text tweet-te...",['https://twitter.com/TreosOfficial/status/121...,...,0.618,0.200,0.1280,2020-01-01,49,13,False,2020-01-01,564.354535,NaN
976,PaylancePH,Paylance Official,3245676818,1212521012408938496,/PaylancePH/status/1212521012408938496,2020-01-01 23:48:31,1577922511,… expect blockchain year 2020 trends shape tec...,"<p class=""TweetTextSize js-tweet-text tweet-te...",['https://www.bbvaopenmind.com/en/economy/fina...,...,1.000,0.000,0.0000,2020-01-01,49,13,False,2020-01-01,564.354535,NaN
655,rudapellini,Rudá Pellini,42966628,1212486577739501574,/rudapellini/status/1212486577739501574,2020-01-01 21:31:41,1577914301,btc bitcoin teve melhor performance da última ...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],...,0.944,0.000,-0.3818,2020-01-01,49,13,False,2020-01-01,564.354535,NaN
218,JosephRockins,Joseph Rockins,806248048242413568,1212437250975375362,/JosephRockins/status/1212437250975375362,2020-01-01 18:15:41,1577902541,click see 24-hour bitcoin price prediction bet...,"<p class=""TweetTextSize js-tweet-text tweet-te...","['http://Pynk.io', 'https://beta.pynk.io/p/Sur...",...,1.000,0.000,0.0000,2020-01-01,49,13,False,2020-01-01,564.354535,NaN


In [33]:
btc_merged.columns

Index(['screen_name', 'username', 'user_id', 'tweet_id', 'tweet_url',
       'timestamp', 'timestamp_epochs', 'text', 'text_html', 'links',
       'hashtags', 'has_media', 'img_urls', 'video_url', 'likes', 'retweets',
       'replies', 'is_replied', 'is_reply_to', 'parent_tweet_id',
       'reply_to_users', 'neg', 'neu', 'pos', 'compound', 'date', 'Bitcoin',
       'BTC', 'isPartial', 'date_vcrix', 'vcrix', 'vcrix_log_diff'],
      dtype='object')

In [34]:
btc_merged.tail()

,screen_name,username,user_id,tweet_id,tweet_url,timestamp,timestamp_epochs,text,text_html,links,...,neu,pos,compound,date,Bitcoin,BTC,isPartial,date_vcrix,vcrix,vcrix_log_diff
992,BittrexPrices,BittrexPrices,1000111792444657665,1212522980854632450,/BittrexPrices/status/1212522980854632450,2020-01-01 23:56:20,1577922980,prices update usdt 1 hour btc 7198.97 +0.07 et...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],...,1.000,0.000,0.0000,2020-01-01,49,13,False,2020-01-01,564.354535,NaN
993,domainrecap,Domain Recap,938629455446200325,1212523382463438849,/domainrecap/status/1212523382463438849,2020-01-01 23:57:56,1577923076,released tbr .ca results leftovers january 1 b...,"<p class=""TweetTextSize js-tweet-text tweet-te...",['https://www.domainrecap.com/2020/picks/to-be...,...,1.000,0.000,0.0000,2020-01-01,49,13,False,2020-01-01,564.354535,NaN
994,Zectro1,Zectro,1035701087167102976,1212523515632586756,/Zectro1/status/1212523515632586756,2020-01-01 23:58:28,1577923108,'ve got divorce idea every critic bsv fan btcn...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],...,0.481,0.225,-0.4534,2020-01-01,49,13,False,2020-01-01,564.354535,NaN
995,reqBitcoinJS,Bitcoin.js //Bitcoin price every half hour,1082323663238713345,1212523651477786624,/reqBitcoinJS/status/1212523651477786624,2020-01-01 23:59:00,1577923140,"jan 1 2020 23:58:00 utc 7,188 usd €6,409 eur £...","<p class=""TweetTextSize js-tweet-text tweet-te...",[],...,1.000,0.000,0.0000,2020-01-01,49,13,False,2020-01-01,564.354535,NaN
996,bitcointe,Bitcointe 💎,1111726282008641542,1212523680120627200,/bitcointe/status/1212523680120627200,2020-01-01 23:59:07,1577923147,key gamers crypto blockchain share new 12 mont...,"<p class=""TweetTextSize js-tweet-text tweet-te...",['https://www.bitcointe.com/2020/01/01/key-gam...,...,0.774,0.226,0.3612,2020-01-01,49,13,False,2020-01-01,564.354535,NaN


In [36]:
btc_merged.to_csv('../data/btc_final.csv', index=False)

### Aggregation by day

In [37]:
btc_merged.columns

Index(['screen_name', 'username', 'user_id', 'tweet_id', 'tweet_url',
       'timestamp', 'timestamp_epochs', 'text', 'text_html', 'links',
       'hashtags', 'has_media', 'img_urls', 'video_url', 'likes', 'retweets',
       'replies', 'is_replied', 'is_reply_to', 'parent_tweet_id',
       'reply_to_users', 'neg', 'neu', 'pos', 'compound', 'date', 'Bitcoin',
       'BTC', 'isPartial', 'date_vcrix', 'vcrix', 'vcrix_log_diff'],
      dtype='object')

In [38]:
btc_by_day = btc_merged.resample('d', on='date').mean().dropna(how='all')

In [39]:
print(btc_merged.shape, btc_by_day.shape)

(997, 32) (1, 20)


In [40]:
btc_merged[btc_merged['compound'].isnull()]

,screen_name,username,user_id,tweet_id,tweet_url,timestamp,timestamp_epochs,text,text_html,links,...,neu,pos,compound,date,Bitcoin,BTC,isPartial,date_vcrix,vcrix,vcrix_log_diff
0,BinancePrices,BinancePrices,1000390115611566080,1212415970423259136,/BinancePrices/status/1212415970423259136,2020-01-01 16:51:07,1577897467,prices update usdt 1 hour btc 7227.23 +0.1 eth...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],...,NaN,NaN,NaN,2020-01-01,49,13,False,2020-01-01,564.354535,NaN


In [42]:
btc_by_day.head()

,user_id,tweet_id,timestamp_epochs,has_media,video_url,likes,retweets,replies,is_replied,is_reply_to,parent_tweet_id,neg,neu,pos,compound,Bitcoin,BTC,isPartial,vcrix,vcrix_log_diff
date,,,,,,,,,,,,,,,,,,,,
2020-01-01,6.313897e+17,1.212470e+18,1.577910e+09,0.222668,NaN,3.223671,0.968907,0.887663,0.160481,0.135406,1.205296e+18,0.023848,0.882992,0.092149,0.186096,49.0,13.0,0.0,564.354535,NaN
